In [2]:
from datasets import load_dataset
ashaar = load_dataset('Zaid/ashaar_dataset')

Found cached dataset parquet (/home/alyafey22/.cache/huggingface/datasets/Zaid___parquet/Zaid--ashaar_dataset-9431b392390e38a6/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
ashaar['train'][0]['text']

'<|meter_7|> <|psep|> <|bsep|> أَصبَحَ المُلك لِلَّذي فَطر الخَل <|vsep|> قَ بِتَقديرٍ للعَزيز العَليمِ </|bsep|> <|bsep|> غافر الذَنب للمسيءِ بِعَفوٍ <|vsep|> قابل التَوب ذي العَطاء العَميمِ </|bsep|> <|bsep|> مُرسل المُصطَفى البَشير ِلَينا <|vsep|> رَحمة مِنهُ بِالكَلام القَديمِ </|bsep|> <|bsep|> رَبَنا رَبّنا ِلَيكَ أَنينا <|vsep|> فَأَجرنا مِن حَر نار الجَحيمِ </|bsep|> <|bsep|> وَاكفِنا شَرّ ما نَخاف بِلُطفٍ <|vsep|> يا عَظيماً يَرجى لِكُل عَظيمِ </|bsep|> <|bsep|> وَتَقبل أَعمالَنا وَاعفُ عَنا <|vsep|> وَأَنلنا دُخول دار النَعيمِ </|bsep|> <|bsep|> بِنَبي بَعثَتهُ فَهَدانا <|vsep|> لِصِراط مِن الهُدى مُستَقيمِ </|bsep|> <|bsep|> وَبِمَن نَحنُ في حِماهُ مَدى الدَهر <|vsep|> أَخيهِ يَحيى الحصور الكَريمِ </|bsep|> <|bsep|> أَدرك أَدرك قَوماً أَتوا بافتقار <|vsep|> وَاِنكِسار وَمَدمَع مَسجومِ </|bsep|> </|psep|>'

In [4]:
ashaar

DatasetDict({
    train: Dataset({
        features: ['poem title', 'poem meter', 'poem verses', 'poem theme', 'poem url', 'poet name', 'poet description', 'poet url', 'poet era', 'poet location', 'poem description', 'poem language type', 'text'],
        num_rows: 144188
    })
})

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
gpt_tokenizer = AutoTokenizer.from_pretrained('Zaid/ashaar_tokenizer')
encoded_input = gpt_tokenizer(['<|psep|>'], return_tensors='pt')
print(encoded_input)

{'input_ids': tensor([[1]]), 'attention_mask': tensor([[1]])}


In [6]:
meter_to_token = {'المديد': '<|meter_0|>', 'السريع': '<|meter_1|>', 'الرجز': '<|meter_2|>', 'الكامل': '<|meter_3|>', 'المتقارب': '<|meter_4|>', 'المجتث': '<|meter_5|>', 'المضارع': '<|meter_6|>', 'الخفيف': '<|meter_7|>', 'الرمل': '<|meter_8|>', 'البسيط': '<|meter_9|>', 'الهزج': '<|meter_10|>', 'الطويل': '<|meter_11|>', 'المنسرح': '<|meter_12|>', 'المقتضب': '<|meter_13|>', 'المتدارك': '<|meter_14|>', 'الوافر': '<|meter_15|>'}
token_to_meter = {t:m for m,t in meter_to_token.items()}

In [37]:
model = AutoModelForCausalLM.from_pretrained('gpt_results_fix_tokens/checkpoint-25000')
text = "<|meter_2|>"
encoded_input = gpt_tokenizer(text, return_tensors='pt')
output = model.generate(**encoded_input, max_length = 512, top_p = 3, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [38]:
result = ""
for i, beam in enumerate(output):
    for token in beam:
        decoded = gpt_tokenizer.decode(token)
        # print(decoded)
        if 'meter' in decoded:
            result += 'قصيدة من بحر '+token_to_meter[decoded] 
        elif decoded == "<|vsep|>":
            result += "*** "
        elif decoded in ["<|bsep|>", "</|bsep|>"]:
            result += "\n"
        elif decoded in ['<|psep|>', '</|psep|>']:
            print()
        else:
            result += decoded
print(result)


قصيدة من بحر الرجز
عجبتُ مِنه بجانحةٍ *** للمصطفى عن رِضا سَحرِ 

مع روضةٍ جَرَحَت وفا *** راحة مع وردةِ النَضرِ 

وزهرةٍ زهراءِ مشرقةٍ *** كالزهر في يومِ النَمرِ 

وردٍ بعينٍ مسفرة *** تجلو ذيال الُعُصفُرِ المُضرِ 

كالغصنِ في وجناتِه قَبَساً *** تُنَشِّق العجابَ بالنَكرِ 

أرددتَهُ نَظرْتَه مهجاً *** والياسمين كلّ مقتدرِ 

لَقَد عرفتُ سَوامه حُجَراً *** مثل الغصون على قَمرِ 

وزهرةِ عَدنانٍ بدَغتها *** بدرّةٍ في بَدَرٍ نُضْرِ 

كأنما الغُدوانُ رائقةٌ *** صبّاغةٌ عن غُصنها القَمري 

وكرّة فكرة جاذبة *** تمهي وتسقي ضيغمَ الثشرِ 

وافى على صوت الصَّبى ش
